In [1]:
# รายชื่อไฟล์ input และ output สำหรับ Papermill
input_path_nov = '../data/raw/2019-Nov-sample-10k.csv'
output_rfm_csv = '../data/processed/rfm_results.csv'

# **Data Preprocessing**

In [2]:
#Import Library
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import squarify

# **Load The Data**

In [3]:
data_nov = pd.read_csv(input_path_nov,usecols=['event_time','event_type','price','user_id','user_session'])

In [4]:
data_nov.head()

,event_time,event_type,price,user_id,user_session
0,2019-11-23 12:11:06 UTC,view,406.05,573554817,08a28fb0-ca22-498e-b4a7-cb72b624fa57
1,2019-11-13 04:59:07 UTC,view,326.13,514164869,45759925-e9a4-9980-387f-444c98c3ac1b
2,2019-11-30 09:58:12 UTC,cart,284.95,536564829,d5660a56-ae30-4154-b50a-9fdac31049c6
3,2019-11-18 18:23:04 UTC,view,144.66,534821312,eec4b0b1-49d8-4500-a85e-1bddaa4594a6
4,2019-11-22 13:58:34 UTC,view,231.38,557080193,cbb73efb-5983-4afe-a215-ae1c52b50c36


In [5]:
data_nov['event_type'].unique()

<ArrowStringArray>
['view', 'cart', 'purchase']
Length: 3, dtype: str

In [6]:
data_nov=data_nov.loc[data_nov.event_type == 'purchase']

In [7]:
data = data_nov.copy()
data.head()

,event_time,event_type,price,user_id,user_session
189,2019-11-13 17:02:39 UTC,purchase,56.37,570423846,872d0f51-8cda-4e8a-bf72-3739f9ac070c
199,2019-11-19 07:44:52 UTC,purchase,347.47,573347666,00a7d5f3-c535-4dd4-8532-6c228dc1c0e9
207,2019-11-09 08:39:27 UTC,purchase,174.26,566031367,6b3c8574-9ffd-4409-8996-1313bfa48816
300,2019-11-17 08:15:02 UTC,purchase,411.83,550107843,5879632d-1047-4052-800a-2d45df4d0abd
577,2019-11-28 08:50:43 UTC,purchase,500.37,561841530,e8edafc5-c5e0-4d14-8599-c8302cb0097f


In [8]:
%%time
data['event_time']=pd.to_datetime(data['event_time']).dt.tz_convert(None)

CPU times: user 113 ms, sys: 58.4 ms, total: 172 ms
Wall time: 346 ms


In [9]:
data.dtypes

event_time      datetime64[us]
event_type                 str
price                  float64
user_id                  int64
user_session               str
dtype: object

In [10]:
%%time
#save dataframe as feather in case our notebook got crashed
#feather save column data types
import pyarrow.feather as feather
os.makedirs('tmp', exist_ok=True)  # Make a temp dir for storing the feather file
feather.write_feather(data, './tmp/data')

CPU times: user 10.5 ms, sys: 22.9 ms, total: 33.4 ms
Wall time: 1.05 s


In [11]:
%%time
#load the feather data cause feather more lightweight
data = pd.read_feather('./tmp/data')
data

CPU times: user 5.27 ms, sys: 12.1 ms, total: 17.4 ms
Wall time: 36.3 ms


,event_time,event_type,price,user_id,user_session
189,2019-11-13 17:02:39,purchase,56.37,570423846,872d0f51-8cda-4e8a-bf72-3739f9ac070c
199,2019-11-19 07:44:52,purchase,347.47,573347666,00a7d5f3-c535-4dd4-8532-6c228dc1c0e9
207,2019-11-09 08:39:27,purchase,174.26,566031367,6b3c8574-9ffd-4409-8996-1313bfa48816
300,2019-11-17 08:15:02,purchase,411.83,550107843,5879632d-1047-4052-800a-2d45df4d0abd
577,2019-11-28 08:50:43,purchase,500.37,561841530,e8edafc5-c5e0-4d14-8599-c8302cb0097f
...,...,...,...,...,...
9641,2019-11-26 07:55:25,purchase,495.20,571346468,67615c09-a2db-4aa2-aee2-bdaa191f466c
9703,2019-11-05 16:50:51,purchase,131.66,515357530,4f1be00a-8a3e-4a65-b5a6-322467f62440
9709,2019-11-04 02:29:45,purchase,241.86,563512526,5c9cfc95-80bd-40ec-91c7-8e955aadc01c
9815,2019-11-01 12:54:13,purchase,77.20,513394156,e6a3232f-5201-494f-9a60-f6ab8a117407


In [15]:
data=data.groupby(by='user_session').agg(Date_order=('event_time',lambda x: x.max()),
                                                  user_id=('user_id', 'first'),
                                          Quantity=('user_session','count'),
                                         money_spent=('price','sum')).reset_index(drop=True)
data

,Date_order,user_id,Quantity,money_spent
0,2019-11-19 07:44:52,573347666,1,347.47
1,2019-11-07 09:30:49,538333338,1,41.19
2,2019-11-24 19:06:44,576118696,1,490.41
3,2019-11-12 21:50:13,543032427,1,119.67
4,2019-11-27 09:52:11,547200022,1,125.85
...,...,...,...,...
126,2019-11-26 07:35:23,543320288,1,818.53
127,2019-11-21 22:03:48,560604878,1,86.69
128,2019-11-17 16:11:53,512936745,1,190.46
129,2019-11-12 06:04:04,567996919,1,970.54


# **RFM Analysis**

In [16]:
data['Date_order'].max()

Timestamp('2019-11-30 06:49:34')

In [19]:
import datetime as dt
study_date = dt.datetime(2019,12,1)
data=pd.DataFrame(data)
data['last_purchase']=study_date - data['Date_order']
data['last_purchase'] = (data['last_purchase'] / np.timedelta64(1, 'D')).astype(int)
data.head()


,Date_order,user_id,Quantity,money_spent,last_purchase
0,2019-11-19 07:44:52,573347666,1,347.47,11
1,2019-11-07 09:30:49,538333338,1,41.19,23
2,2019-11-24 19:06:44,576118696,1,490.41,6
3,2019-11-12 21:50:13,543032427,1,119.67,18
4,2019-11-27 09:52:11,547200022,1,125.85,3


In [20]:
#Calculate Recency, Frequency, and Monetary of the data
RFM= data.groupby('user_id').agg(Recency=('last_purchase',lambda x: x.min()),
                                 Frequency=('user_id',lambda x: x.count()),
                                 Monetary=('money_spent',lambda x: x.sum()))
RFM.head()


,Recency,Frequency,Monetary
user_id,,,
512446130,20,1,36.04
512497797,27,1,69.44
512512888,17,1,694.33
512571717,24,1,22.65
512611827,7,1,565.01


# **Frequency**

Frequency attribute answers the question: How often do they purchase?

In [21]:
RFM['Frequency'].describe()

count    131.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: Frequency, dtype: float64

# **Monetary**

Monetary attribute answers the question: How much do they spend overtime?


In [22]:
RFM['Recency'].describe()

count    131.000000
mean      14.206107
std        7.786297
min        0.000000
25%        7.000000
50%       13.000000
75%       20.500000
max       29.000000
Name: Recency, dtype: float64

# **RFM Segmentation**

**RFM Quartiles**

In [23]:
RFM.quantile(q=[0.25,0.5,0.75])

,Recency,Frequency,Monetary
0.25,7.0,1.0,93.095
0.50,13.0,1.0,167.030
0.75,20.5,1.0,355.850


In [24]:
quartiles=RFM.quantile(q=[0.25,0.5,0.75]).to_dict()
quartiles

{'Recency': {0.25: 7.0, 0.5: 13.0, 0.75: 20.5},
 'Frequency': {0.25: 1.0, 0.5: 1.0, 0.75: 1.0},
 'Monetary': {0.25: 93.095, 0.5: 167.03, 0.75: 355.85}}

# **Creation of RFM Segments**

In [25]:
## for Recency 

def R(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
## for Frequency and Monetary 

def FM(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1    
    

In [26]:
#create RFM segmentation column
RFM['R_Quartile'] = RFM['Recency'].apply(R, args=('Recency',quartiles,))
RFM['F_Quartile'] = RFM['Frequency'].apply(FM, args=('Frequency',quartiles,))
RFM['M_Quartile'] = RFM['Monetary'].apply(FM, args=('Monetary',quartiles,))
RFM['RFM_segmentation'] = RFM.R_Quartile.map(str) \
                    + RFM.F_Quartile.map(str) \
                    + RFM.M_Quartile.map(str)
RFM['RFM_score'] = RFM.R_Quartile.map(int) \
                    + RFM.F_Quartile.map(int) \
                    + RFM.M_Quartile.map(int)
RFM.head()


,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFM_segmentation,RFM_score
user_id,,,,,,,,
512446130,20,1,36.04,3,4,4,344,11
512497797,27,1,69.44,4,4,4,444,12
512512888,17,1,694.33,3,4,1,341,8
512571717,24,1,22.65,4,4,4,444,12
512611827,7,1,565.01,1,4,1,141,6


In [27]:
# Define rfm_level function
def RFM_label(data):
    if data['RFM_score'] >= 10:
        return 'Lost'
    elif ((data['RFM_score'] >= 9) and (data['RFM_score'] < 10)):
        return 'Hibernating'
    elif ((data['RFM_score'] >= 8) and (data['RFM_score'] < 9)):
        return 'Can’t Lose Them'
    elif ((data['RFM_score'] >= 7) and (data['RFM_score'] < 8)):
        return 'About To Sleep'
    elif ((data['RFM_score'] >= 6) and (data['RFM_score'] < 7)):
        return 'Promising'
    elif ((data['RFM_score'] >= 5) and (data['RFM_score'] < 6)):
        return 'Potential Loyalist'
    elif ((data['RFM_score'] >= 4) and (data['RFM_score'] < 5)):
        return 'Loyal Customers'
    else:
        return 'Champions'
#Create RFM label for customer
RFM['RFM_label'] = RFM.apply(RFM_label, axis=1)
RFM.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFM_segmentation,RFM_score,RFM_label
user_id,,,,,,,,,
512446130,20,1,36.04,3,4,4,344,11,Lost
512497797,27,1,69.44,4,4,4,444,12,Lost
512512888,17,1,694.33,3,4,1,341,8,Can’t Lose Them
512571717,24,1,22.65,4,4,4,444,12,Lost
512611827,7,1,565.01,1,4,1,141,6,Promising


In [28]:
# บันทึกไฟล์เพื่อส่งต่อให้ output.ipynb
RFM.to_csv(output_rfm_csv)
print(f"Analysis completed. Results saved to {output_rfm_csv}")

Analysis completed. Results saved to ../data/processed/rfm_results.csv
